# Context Engineering: LLMs for LF

## Objective
Generate Lingua Franca (LF) code from natural language prompt.

## Context
Large Language Models (LLMs) are a powerful tool for generating code. However, LLMs are often trained on standard programming languages like C/C++, Java, Python, and JavaScript. This makes it difficult to generate code in Domain-specific languages like Lingua Franca.

This project shows how to curate context to guide the LLM.  We demonstrate the technique by showing how to generate code that uses the Lingua Franca coordination language together with C and Python.

## 1. LF code generation with manually engineered context

### 1.1 Environment Setup:
- Virtual environment installation.
- Necessary libraries installation.
- Necessary imports.
- OPEN_AI_API_KEY loading.

In [ ]:
# Installing Required Packages
!python -m pip install --upgrade pip
!python -m pip install python-dotenv openai

In [125]:
# Importing Libraries
import os
from dotenv import load_dotenv
from openai import OpenAI

A `.env` file is used to store sensitive information that you don't want to hard-code into your source code.


For this project `OPENAI_API_KEY` is stored in `.env` file as:


`OPENAI_API_KEY='your_openai_api_key'`



In [126]:
# Loading OPENAI_API_KEY from the .env file
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.environ.get('OPENAI_API_KEY')

### 1.2 OpenAI models


| Model | context length | Input | Output | Training Data|
| :--- | :--- | :--- | :--- | :--- | 
| GPT-4o |  128k | \$5  | \$15  | Up to Oct 2023|
| GPT-4 Turbo |  128k  |  \$10 | \$30  |Up to Dec 2023|
| GPT-3.5 Turbo |  16k  |  \$0.50 |  \$1.50| Up to Sep 2021


\* prices per 1 million tokens.
\* 100 tokens ~= 75 words.

The foolowing C++ line (10 words):

`for (int i = 0; i < g[x].size(); i++) {⤶`

is composed of 23 tokens:

~~~~
for 
(
i
nt 
i
 
= 
0; 
i
 
<
 
g
[
x
]
.s
ize(
); 
i
+
+) 
{⤶
~~~~

- A 50 lines C++ code ~= 600 tokens. Which costs 0.3 cents.

### 1.3 Prompting OpenAI models
#### 1.3.1 Without context
We start by prompting OpenAI LLMs: `gpt-3.5` and `gpt-4o` without any additional context.

The generated code will be stored under `LFGPT/without_context`.

We start by creating the target directory.

In [127]:
# Creating the target directory
!mkdir -p LFGPT/without_context

Introduce a `run_iterator` variable to prevent overwriting the generated code during multiple executions.

Define the prompt for a DropSensor reactor.

![DropSensor sample](img/DropSensor.png)

- Triggers the accelerometer with period of 250 ms.
- If the absolute values of the accelerometer readings (`x`, `y`, `z`) are all less than the defined `threshold`, then `print(x, y, z)`.


In [128]:

# Defining the prompt
prompt = f""" 
  Provide a Lingua Franca code for 'target C'. 
  Import the LF 'Accelerometer' reactor from library.
  Create a main reactor named `DropSensor` that:
  - has a parameter named `threshold`, with a default value of '0.3'.
  - instantiates an accelerometer.
  - declares a timer with a period of 250 ms.
  - Has a reaction that triggers the accelerometer with period of 250 ms by setting the accelerometer trigger to true.
  - has another reaction which fires whenever any of the accelerometer readings (`x`, `y`, or `z`) change. Inside this reaction: It checks whether the absolute values of the accelerometer readings (`x`, `y`, `z`) are all less than the defined `threshold`. If the condition is true, it prints the values of the accelerometer outputs to the console.  
  """
# File name suffix
file_name = "LFGPT/without_context/DropSensor"

# Defining run iterartor
run_iterator = 0


In [130]:
# Initializes an OpenAI client for interacting with the OpenAI API.
client = OpenAI()


MODELS = ["gpt-4o", "gpt-3.5-turbo"] 
run_iterator += 1

for MODEL in MODELS:
  fout = open(f'{file_name}_{MODEL}_{run_iterator}.lf', 'w')
  
  completion = client.chat.completions.create(
    model=MODEL,
    messages=[
      {"role": "system", "content": prompt}
    ]
  )

  print(completion.choices[0].message.content, file=fout)
  fout.close()

  print(f'{file_name}_{MODEL}_{run_iterator}.lf')


LFGPT/without_context/DropSensor_gpt-4o_2.lf
LFGPT/without_context/DropSensor_gpt-3.5-turbo_2.lf


#### 1.3.2 With manual context
We start by prompting OpenAI LLMs: `gpt-3.5` and `gpt-4o` with handcrafted context.

The generated code will be stored under `LFGPT/with_context`.

We start by creating the target directory.

In [131]:
# Creating the target directory
!mkdir -p LFGPT/with_context

In [132]:

# Defining the context
context = r"""
  A reactor is a software component that reacts to input events, timer events, and internal events. It has private state variables that are not visible to any other reactor. Its reactions can consist of altering its own state, sending messages to other reactors, or affecting the environment through some kind of actuation or side effect (e.g., printing a message, as in the above HelloWorld example).
  The general structure of a reactor definition is as follows:
  [main or federated] reactor <class-name> [(parameters)] {
    input <name>: <type>
    output <name>: <type>
    state <name>: <type> [= <value>]
    timer <name>([<offset>[, <period>]])
    logical action <name>[: <type>]
    physical action <name>[: <type>]
    reaction [<name>] (triggers) [<uses>] [-> <effects>] [{= ... body ...=}]
    <instance-name> = new <class-name>([<parameter-assignments>])
    <port-name> [, ...] -> <port-name> [, ...] [after <delay>]
}
  """


# Defining run iterartor
run_iterator = 0


# File name suffix
file_name = "LFGPT/with_context/DropSensor"

In [133]:
# Initializes an OpenAI client for interacting with the OpenAI API.
client = OpenAI()


MODELS = ["gpt-4o", "gpt-3.5-turbo"] 
run_iterator += 1

for MODEL in MODELS:
  fout = open(f'{file_name}_{MODEL}_{run_iterator}.lf', 'w')
  
  completion = client.chat.completions.create(
    model=MODEL,
    # Adding context to the prompt
    messages=[
      {"role": "system", "content": context + prompt}
    ]
  )

  print(completion.choices[0].message.content, file=fout)
  fout.close()


## 2. LF code generation with automatically Retrieved context
Now we will use Retrieval-Augmented Generation (RAG). RAG combines retrieval-based methods with generative models to improve the quality and relevance of generated code.

![Retrieval-Augmented Generation (RAG)](img/RAG.png)

### 2.1 Retreival Model
### Embedding
A vector embedding, also called embedding, is a numerical representation of the semantics, or meaning of a text. 



<img src="img/embedding.png" alt="Embedding sample" style="width: 1000px;"/>


Two pieces of text with similar meanings will have mathematically similar embeddings, even if the actual text is quite different.

We will use LlamaIndex [https://www.llamaindex.ai/] to generate the embedding of each file in our codebase [https://github.com/lf-lang/playground-lingua-franca]. 

#### How it works?
- LlamaIndex splits files as `nodes`, 
- creates the embedding of each `nodes` using a LLM,
- and stores it in a `VectorStoreIndex`.
- We then request the `VectorStoreIndex` to retrieve the `nodes` that are most similar to the user prompt.




In [134]:
# Install llama-index
!python -m pip install llama-index

In [ ]:
# Importing packages

from llama_index.core import (
    load_index_from_storage,
    SimpleDirectoryReader,
    StorageContext,
    VectorStoreIndex
    
)

# Setting codebase directory
# contains examples directory from https://github.com/lf-lang/playground-lingua-franca
DIR = "./all_codebase"

# Setting storage directory
PERSIST_DIR = "./index_all_codebase"

In [135]:
# If `VectorStoreIndex` exists load it from disk.
# Else, Build it from code_base and save it to disk, for future use.

def load_storage_context():
    # if 'PERSIST_DIR' exists, load indexes from persisted data
    # else load create indexes from code base dierctory, 'data'.
    if (os.path.exists(PERSIST_DIR)):
        # load the existing index
        print("Loading persisted indexes ...")
        # rebuild storage context
        storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
        return storage_context      
    else:
        print("Creating indexes from scratch ...")
        # load documents from directory
        documents = SimpleDirectoryReader(DIR, recursive=True).load_data()

        # build index with embedding model "gpt-4o"
        index = VectorStoreIndex.from_documents(documents, model="gpt-4o")

        # save indexes on disk
        index.storage_context.persist(persist_dir=PERSIST_DIR) 
        return None

# Loading `VectorStoreIndex` (storage context) from disk
storage_context = load_storage_context()


Loading persisted indexes ...


### 2.2 Retrieval-Augmented Generation

From here we only focus on `gpt-4o` model.

The generated code will be stored under `LFGPT/rag`.

We start by creating the target directory.

In [136]:
# Creating the target directory
!mkdir -p LFGPT/rag

In [149]:
from llama_index.llms.openai import OpenAI

# Define LLM to use for Code geneartion
MODEL = "gpt-4o"

# Define the Top K retrived documents (LF files)
TOP_K = 10

# Instatntiate the LLM
llm = OpenAI(temperature=1, model=MODEL)

# Instantiate a ServiceContext using the OpenAI_API_key
index = load_index_from_storage(storage_context, llm=llm)        
 
# Configure retriever
query_engine = index.as_query_engine(similarity_top_k=TOP_K)



# Defining run iterartor
run_iterator = 0

# File name suffix
file_name = "LFGPT/rag/DropSensor"

In [152]:
# Defining the prompt
prompt = f""" 
  Provide a Lingua Franca code for 'target Python'. 
  Import the LF 'Accelerometer' reactor from library.
  Create a main reactor named `DropSensor` that:
  - has a parameter named `threshold`, with a default value of '0.3'.
  - instantiates an accelerometer.
  - declares a timer with a period of 250 ms.
  - Has a reaction that triggers the accelerometer with period of 250 ms by setting the accelerometer trigger to true.
  - has another reaction which fires whenever any of the accelerometer readings (`x`, `y`, or `z`) change. Inside this reaction: It checks whether the absolute values of the accelerometer readings (`x`, `y`, `z`) are all less than the defined `threshold`. If the condition is true, it prints the values of the accelerometer outputs to the console.  
  """

In [139]:

def rag():    
    response = query_engine.query(prompt)
    files = '/**\n'
    for node in response.source_nodes:
        files += f'* {node.metadata["file_path"]}\n'
    files += '*/\n'

    fout = open(f'{file_name}_{run_iterator}_K-{TOP_K}.lf', 'w')

    print(files+response.response, file=fout)
    fout.close()

    print(f'{file_name}_{run_iterator}_K-{TOP_K}.lf')




In [143]:
run_iterator += 1
rag()

LFGPT/rag/DropSensor_3_K-10.lf


### Python target with index composed of Python only files

In [144]:
# Setting codebase directory
# contains Python examples from :
# 1. https://github.com/lf-lang/playground-lingua-franca
# 2. https://github.com/lf-lang/lf-lang.github.io
DIR = "./python_codebase"

# Setting storage directory
PERSIST_DIR = "./index_python_codebase"

# Loading `VectorStoreIndex` (storage context) from disk
storage_context = load_storage_context()

Loading persisted indexes ...


In [153]:
# Instantiate a ServiceContext using the OpenAI_API_key
index = load_index_from_storage(storage_context, llm=llm)        
 
# Configure retriever
query_engine = index.as_query_engine(similarity_top_k=TOP_K)



In [154]:
run_iterator += 1
rag()

LFGPT/rag/DropSensor_3_K-10.lf
